In [ ]:
!pip install transformers text-hammer pyreadstat

In [ ]:
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
import torch.nn.functional as F
from transformers import RobertaTokenizer, RobertaForSequenceClassification, AdamW, get_linear_schedule_with_warmup , AutoTokenizer, TFAutoModel
import numpy as np
import seaborn as sns
from sklearn.metrics import confusion_matrix,classification_report,accuracy_score, f1_score
import matplotlib.pyplot as plt
import os
import io
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, KFold
from sklearn import svm
from collections import defaultdict
import text_hammer as th
import pandas as pd
import tensorflow as tf , keras
import transformers
import random as rd
import keras.backend as K
from numpy.random import seed
from tensorflow.keras import layers
from keras.utils import plot_model
from keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn import metrics
from tensorflow.keras.models import load_model
import warnings
warnings.filterwarnings("ignore")
%matplotlib inline

In [ ]:
# identify and specify the GPU as the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()

In [ ]:
def compute_validation_loss(model, validation_dataloader):
    model.eval()
    total_val_loss = 0
    with torch.no_grad():
        for batch in validation_dataloader:
            input_ids, attention_mask, labels = batch
            input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)

            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            total_val_loss += loss.item()

    average_val_loss = total_val_loss / len(validation_dataloader)
    return average_val_loss

def metric2(y_true, y_pred_classes):
    n = y_true.size(0)
    # Calculate errors where prediction is off by 1 class
    res = torch.abs(y_true - y_pred_classes)
    count_error = torch.sum(res == 1, dtype=torch.float32)
    metric = 1 - count_error / n
    return metric.item()

def metric2_2(y_true, y_pred):
    # Convert to numpy arrays for easier manipulation
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    # Calculate the number of predictions off by 1 class
    off_by_one = np.sum(np.abs(y_true - y_pred) == 1)
    # Calculate the metric
    metric = 1 - off_by_one / len(y_true)
    return metric


In [ ]:
def get_clean(x):
    mention = r'@\w+'
    hash = r'#\w+'
    x = str(x).lower().replace('\\', '').replace('_', '')
    x = re.sub(r'[^\x00-\x7F]+', ' ', x)
    x = th.cont_exp(x)
    x = th.remove_emails(x)
    x = th.remove_urls(x)
    x = re.sub(mention, ' ', x)
    x = re.sub(hash, ' ', x)
    x = th.remove_html_tags(x)
    x = th.remove_rt(x)
    x = th.remove_accented_chars(x)
    x = th.remove_special_chars(x)
    x = re.sub("(.)\\1{2,}", "\\1", x)
    x = re.sub(r'\s+', ' ', x).strip()
    x = re.sub(r'\w*\d+\w*', ' ', x).strip()
    return x

In [ ]:
df = pd.read_spss("/content/drive/MyDrive/VA_EN_TU_2012-2020_3000_tweets_relevant_V03_labeled_1200_cleaned.sav")
# Original labels: 0 - positive, 1 - negative, 2 - neutral
# Remapping dictionary to align with RoBERTa's expected labels
label_mapping = {1: 2, 2: 0, 3: 1}

# Remapped labels: 0 - negative, 1 - neutral, 2 - positive
df['Label_B_emotion'] = df['Label_B_emotion'].replace(label_mapping)
df['Label_B_emotion'] = df['Label_B_emotion'].astype(int)
dff = df[['text','Label_B_emotion']].copy()
dff['Label_B_emotion'].unique()

In [ ]:
dff['cleaned_data'] = dff['text'].apply(get_clean)

In [ ]:
nb_samples = len(dff)
proportion_training = 0.80
pretrained_model_name = "cardiffnlp/twitter-roberta-base-sentiment-latest"
seq_len = 128
batch_size = 40
learning_rate = 2e-5
nb_epoch = 60
nb_fold = 3
earlyStopPatience = 10

In [ ]:
def tokenize(sentence):
    tokens = tokenizer.encode_plus(sentence, max_length=seq_len,
                                   truncation=True, padding='max_length',
                                   add_special_tokens=True, return_attention_mask=True,
                                   return_token_type_ids=False, return_tensors='tf')
    return tokens['input_ids'], tokens['attention_mask']

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name)

In [ ]:
X_ids = np.zeros((len(dff),seq_len))
X_mask = np.zeros((len(dff),seq_len))

for i, sentence in enumerate(dff['text']):
    X_ids[i, :], X_mask[i, :] = tokenize(sentence)

In [ ]:
sep = round(proportion_training * len(dff))
labels = dff[['Label_B_emotion']].values

X_ids_train = X_ids[:sep,:]
X_mask_train = X_mask[:sep,:]
Y_train = labels[:sep,:]

X_ids_test = X_ids[sep:,:]
X_mask_test = X_mask[sep:,:]
Y_test = labels[sep:,:]

In [ ]:
def metric1(y_true, y_pred):
    n = K.cast(K.shape(y_true)[0], 'float32')
    y_pred = K.reshape(y_pred,(n,1))
    res = K.equal(y_true,y_pred)
    res = K.cast(res, 'float32')
    res = K.sum(res) / n
    return res

In [ ]:
pretrained_model = TFAutoModel.from_pretrained(pretrained_model_name)

In [ ]:
def get_model():

    input_text_ids = keras.Input(shape=(seq_len,), dtype='int32', name='input_text_ids')
    text_mask = keras.Input(shape=(seq_len,), dtype='int32', name='attention_mask_text')

    embeddings = pretrained_model(input_text_ids, attention_mask=text_mask)[1]

    x = tf.keras.layers.Dense(512, activation='relu')(embeddings)
    x = tf.keras.layers.Dropout(0.5)(x)
    x = tf.keras.layers.Dense(128, activation='relu')(x)
    x = tf.keras.layers.Dropout(0.5)(x)

    predictions = layers.Dense(3, activation="softmax", name="predictions")(x)

    model = keras.Model(inputs=[input_text_ids,text_mask], outputs=predictions)

    model.layers[2].trainable = False

    model.compile(optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
                loss="sparse_categorical_crossentropy",
                metrics="accuracy")
    return model

In [ ]:
model = get_model()
model.summary()

In [ ]:
def fit_the_model(model, X_tr, y_tr, X_val, y_val, num):
    callbacks = [
        EarlyStopping(monitor="val_accuracy", patience=earlyStopPatience, mode="max"),
        ModelCheckpoint(filepath="MODEL/best_model"+str(num)+".hdf5", monitor="val_accuracy", mode='max', save_best_only=True, verbose=1)
    ]
    model.fit(X_tr, y_tr, validation_data = (X_val,y_val), epochs=nb_epoch, verbose=1, callbacks=callbacks, batch_size=batch_size)
    return model

In [ ]:
list_of_models = []
list_history = []

kfold = KFold(n_splits=nb_fold, shuffle=True, random_state=42)

# Training with k-fold method
k = 1
for train_index, val_index in kfold.split(X_ids_train,Y_train):
    print(f'\nTraining model {k}...')
    model = get_model()
    history = fit_the_model(model,
                            [X_ids_train[train_index],
                             X_mask_train[train_index]],
                            Y_train[train_index],
                            [X_ids_train[val_index],
                             X_mask_train[val_index]],
                            Y_train[val_index], k)
    list_of_models.append(history)
    list_history.append(history.history.history)
    k += 1

print("---Finished---")

In [ ]:
def ConfusionMatrix(y_true , y_pred):
    cm = confusion_matrix(y_true , y_pred)
    return cm/y_true.shape[0]

In [ ]:
def metric2_per_class(y_true,y_pred):
    L = []
    y_pred = y_pred.reshape((len(y_pred),1))
    for i in range(0,3):
        L.append(y_true[(y_true==y_pred) & (y_true==i)].shape[0] / y_true[(y_true==i)].shape[0])
    return L

In [ ]:
for i in range(nb_fold):
    print(50*'-',f"MODEL PERFORMANCE {i+1}",50*'-')

    average_loss = list_history[i]['loss']
    average_val_loss = list_history[i]['val_loss']

    average_metric_1 = list_history[i]['accuracy']
    average_val_metric_1 = list_history[i]['val_accuracy']

    plt.figure(figsize=(16,4))

    txtTitres = ["Training and validation loss","Training and validation accuracy"]
    txtYLabel = ["Loss","Metric 1 "]

    L1 = [average_loss, average_metric_1]
    L2 = [average_val_loss, average_val_metric_1]

    for k in range(2):
        if len(L1[0]) < nb_epoch:
            nb_epoch_plot=len(L1[0])
        else:
            nb_epoch_plot=nb_epoch
        Epochs = range(1, nb_epoch_plot+1)
        plt.subplot(1,3,k+1)
        plt.plot(Epochs, L1[k], color='b', marker='o', label="training")
        plt.plot(Epochs, L2[k], color='r', marker='o', label="validation")
        plt.grid()
        plt.title(txtTitres[k])
        plt.xlabel("Epochs")
        plt.ylabel(txtYLabel[k])
        plt.legend()
    plt.show()

In [ ]:
L_best_models = []
for i in range(nb_fold):
    path = "MODEL/best_model" + str(i+1) + ".hdf5"
    L_best_models.append(keras.models.load_model(path,
                                                 custom_objects={"TFRobertaModel": pretrained_model},
                                                 compile=False))

In [ ]:
L_predictions = []
L_cm = []
L_f1 = []
L_m1, L_m2 = [], []

k = 0

kfold = KFold(n_splits=nb_fold, shuffle=True, random_state=42)
for train_index, val_index in kfold.split(X_ids_train,Y_train):

    prediction = np.argmax(L_best_models[k].predict([X_ids_train[val_index], X_mask_train[val_index]]), axis=1)

    cm = ConfusionMatrix(Y_train[val_index] , prediction)
    f1 = f1_score(Y_train[val_index] , prediction, average=None)
    m1 = metric1(Y_train[val_index], prediction)
    m2 = metric2_per_class(Y_train[val_index] , prediction)

    L_predictions.append(prediction)
    L_cm.append(cm)
    L_f1.append(f1)
    L_m1.append(m1)
    L_m2.append(m2)
    k += 1

In [ ]:
print(40*'-',f"SUMMARY OF MODEL PERFORMANCE ON VALIDATION DATA",20*'-',"\n")

m1 = np.mean(L_m1,axis=0)
m2 = np.mean(L_m2,axis=0)

print("metric1 validation mean :", m1, "\n")
print("metric2 validation mean per class :", m2, "\n")

print("Average F1 score on positive/negative/neutral: ", np.mean(L_f1,axis=0))

avg_cm = np.mean(L_cm,axis=0)
plt.figure(figsize=(5,5))
ax = sns.heatmap(avg_cm, annot=True, fmt='.2%' , cmap="Blues")
ax.set_xlabel('\n Predicted Values')
ax.set_ylabel('Actual Values ')
ax.set_title(f'Confusion Matrix of the model ')
ax.xaxis.set_ticklabels(['Negative', 'Neutral' ,'Positive'])
ax.yaxis.set_ticklabels(['Negative', 'Neutral' ,'Positive'])
plt.show()

In [ ]:
L_pred = [np.round(model.predict([X_ids_test, X_mask_test])) for model in L_best_models]
prediction = np.median(L_pred, axis=0)

predicted_labels = np.argmax(prediction, axis=1).reshape(-1, 1)

In [ ]:
print("Classification Report: \n", classification_report(Y_test, predicted_labels,target_names=['Negative','Neutral','Positive']))

In [ ]:
cm = ConfusionMatrix(Y_test , predicted_labels)
f1 = f1_score(Y_test , predicted_labels, average=None)
m1 = float(metric1(Y_test, predicted_labels))
m2 = float(metric2_2(Y_test, predicted_labels))

In [ ]:
print("metric1 Test mean :", m1, "\n")
print("metric2 Test mean :", m2, "\n")
print("Average F1 score on negative/neutral/positive: ", np.mean(f1,axis=0))

plt.figure(figsize=(5,5))
ax = sns.heatmap(cm, annot=True, fmt='.2%' , cmap="Blues")
ax.set_xlabel('\n Predicted Values')
ax.set_ylabel('Actual Values ')
ax.set_title(f'Confusion Matrix of the model ')
ax.xaxis.set_ticklabels(['Negative', 'Neutral' ,'Positive'])
ax.yaxis.set_ticklabels(['Negative', 'Neutral' ,'Positive'])
plt.show()